# Introduction

This notebook demonstrates how to carry out an ordering of a disordered structure using pymatgen.

In [39]:
import warnings
warnings.filterwarnings('ignore')

In [62]:
# Let us start by creating a disordered CuAu fcc structure.
from pymatgen.core import Lattice, Structure

# specie = {"Cu0+": 0.5, "Au0+": 0.5}
# cuau = Structure.from_spacegroup("Fm-3m", Lattice.cubic(3.677), [specie], [[0, 0, 0]])
cuau = Structure.from_file('/content/LMTOF_order_Disorder_transform_to_primitive.cif')
cuau.add_oxidation_state_by_element({"Li": +1, "Mn": +3,"Ti":+4,"O":-2,"F":-1})
print(cuau)
cuau

Full Formula (Li0.6 Ti0.2 Mn0.2 O0.6 F0.4)
Reduced Formula: Li0.6Ti0.2Mn0.2O0.6F0.4
abc   :   2.974374   2.974374   2.974374
angles:  60.000000  60.000000  60.000000
pbc   :       True       True       True
Sites (2)
  #  SP                             a    b    c
---  ---------------------------  ---  ---  ---
  0  Li+:0.6, Ti4+:0.2, Mn3+:0.2  0    0    0
  1  O2-:0.6, F-:0.4              0.5  0.5  0.5


<IPython.core.display.Javascript object>

Note that each site is now 50% occupied by Cu and Au. Because the ordering algorithms uses an Ewald summation to rank the structures, you need to explicitly specify the oxidation state for each species, even if it is 0. Let us now perform ordering of these sites using two methods.

## Method 1 - Using the OrderDisorderedStructureTransformation

The first method is to use the OrderDisorderedStructureTransformation.

In [59]:
from pymatgen.transformations.standard_transformations import SupercellTransformation


supercell_transformation = SupercellTransformation.from_scaling_factors(1, 1, 5)
supercell_structure = supercell_transformation.apply_transformation(cuau)
cuau = supercell_structure

In [60]:
from pymatgen.transformations.standard_transformations import (
    OrderDisorderedStructureTransformation,
)

trans = OrderDisorderedStructureTransformation()

ss = trans.apply_transformation(cuau, return_ranked_list=100)

print(len(ss))

print(ss[0])

AttributeError: Element has no attribute oxi_state!

Note that the OrderDisorderedTransformation (with a sufficiently large return_ranked_list parameter) returns all orderings, including duplicates without accounting for symmetry. A computed ewald energy is returned together with each structure. To eliminate duplicates, the best way is to use StructureMatcher's group_structures method, as demonstrated below.

In [ ]:
from pymatgen.analysis.structure_matcher import StructureMatcher

matcher = StructureMatcher()
groups = matcher.group_structures([d["structure"] for d in ss])
print(len(groups))
print(groups[0][0])

1
Full Formula (Cu2 Au2)
Reduced Formula: CuAu
abc   :   3.677000   3.677000   3.677000
angles:  90.000000  90.000000  90.000000
Sites (4)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Cu0+  0.5  0    0.5
  1  Cu0+  0.5  0.5  0
  2  Au0+  0    0    0
  3  Au0+  0    0.5  0.5


## Method 2 - Using the EnumerateStructureTransformation

If you have enumlib installed, you can use the EnumerateStructureTransformation. This automatically takes care of symmetrically equivalent orderings and can enumerate supercells, but is much more prone to parameter sensitivity and cannot handle very large structures. The example below shows an enumerate of CuAu up to cell sizes of 4.

In [43]:
from pymatgen.transformations.advanced_transformations import (
    EnumerateStructureTransformation,
)

# specie = {"Cu": 0.5, "Au": 0.5}
# cuau = Structure.from_spacegroup("Fm-3m", Lattice.cubic(3.677), [specie], [[0, 0, 0]])

trans = EnumerateStructureTransformation(max_cell_size=100)
ss = trans.apply_transformation(cuau, return_ranked_list=1000)

In [46]:
print(len(ss))
print("cell sizes are %s" % ([len(d["structure"]) for d in ss]))

90
cell sizes are [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]


In [47]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
# The following two lines write all the structures to CIF files with names CuAu_0.cif, CuAu_1.cif, ...
for i, d in enumerate(ss):
    d["structure"].to(filename="/content/drive/MyDrive/LMTOF_ENUMLIB/CIFS/CIF_%d.cif" % i)

In [57]:
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.core import Structure, Lattice
from tabulate import tabulate
data = []
for i in range(len(ss)):
  structure = Structure.from_file(f"/content/drive/MyDrive/LMTOF_ENUMLIB/CIFS/CIF_{i}.cif")
  sga_default = SpacegroupAnalyzer(structure)
  sg_symbol, sg_number,sg_crystal = sga_default.get_space_group_symbol(), sga_default.get_space_group_number(),sga_default.get_crystal_system()
# structure = Structure.from_file("/content/drive/MyDrive/LMTOF_ENUMLIB/CIFS/CIF_%d.cif")
# sga_default = SpacegroupAnalyzer(structure)
# sg_symbol, sg_number = sga_default.get_space_group_symbol(), sga_default.get_space_group_number()

  # print(f"crystal: {sg_crystal}  spacegroup: symbol: {sg_symbol}, number: {sg_number}")

  data.append([sg_crystal,sg_symbol,sg_number])
print(tabulate(data, headers=["crystal","spacegroup","number"]))

crystal       spacegroup      number
------------  ------------  --------
monoclinic    Cm                   8
orthorhombic  Imm2                44
tetragonal    I4mm               107
trigonal      R3m                160
tetragonal    I4mm               107
tetragonal    I4mm               107
monoclinic    Cm                   8
trigonal      R3m                160
orthorhombic  Imm2                44
orthorhombic  Imm2                44
trigonal      R3m                160
tetragonal    I4mm               107
monoclinic    Cm                   8
monoclinic    Cm                   8
monoclinic    Cm                   8
orthorhombic  Imm2                44
tetragonal    I4mm               107
monoclinic    Cm                   8
monoclinic    Cm                   8
tetragonal    I4mm               107
orthorhombic  Imm2                44
monoclinic    Cm                   8
tetragonal    I4mm               107
monoclinic    Cm                   8
trigonal      R3m                160
m

Note that structures with cell sizes ranging from 1-3x the unit cell size is generated.

## Conclusion

This notebook illustrates two basic ordering/enumeration approaches. In general, OrderDisorderedTransformation works better for large cells and is useful if you need just any quick plausible ordering. EnumerateStructureTransformation is more rigorous, but is prone to sensitivity errors and may require fiddling with various parameters.